# Internship work

In [51]:
# importing the necessary libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup
import chardet
import re

In [52]:
main_url = r'http://valoda.ailab.lv/folklora/pasakas/saturs.htm'
main_response = requests.get(main_url)
main_response.encoding = 'windows-1257'
main_soup = BeautifulSoup(main_response.text, 'lxml')
# Extracting the titles
main_titles_part = main_soup.find_all('a', string = True)

stories_number = [10, 23, 23, 62, 32, 16, 16, 27, 41, 63, 101, 108, 63, 70, 70]
# Create a list of lists for each volume's information
volumes_info = []
stories_number_index = 0
# Iterate through titles_part and create inner lists
for i in range(len(stories_number)):
    volume_info = []
    
    for element in main_titles_part[stories_number_index:stories_number_index + stories_number[i]]:
        title_of_chapter = element.text
        url_of_subchapter = f"http://valoda.ailab.lv/folklora/pasakas/{element.get('href')}"
        volume_info.append((url_of_subchapter, title_of_chapter))
    volumes_info.append(volume_info)
    stories_number_index += stories_number[i]

In [53]:
url_of_chapter = ''
title_of_chapter = ''
title_of_subchapter = ''
metadata_of_chapter = ''
full_story_of_chapter = ''
def add_entry_to_list():
    story_table.append((url_of_chapter, title_of_chapter, title_of_subchapter, metadata_of_chapter, full_story_of_chapter))

In [54]:
base_link_site = 'http://valoda.ailab.lv/folklora/pasakas/'
pattern = r'\b\d+\s*\w*\.\s*'

for inner_list_index in range(len(volumes_info)):
    story_table = []
    for link, title in volumes_info[inner_list_index]:
        story_and_metadata_section = []
        subchapter_response = requests.get(link)
        subchapter_response.encoding = 'windows-1257'
        subchapter_soup = BeautifulSoup(subchapter_response.text, 'lxml')
        
        if subchapter_soup.find_all('a'):
            informations_about_subchapters = subchapter_soup.find_all('a')
            
            for information in informations_about_subchapters:
                
                story_and_metadata_section_wit_sub = []
                title_of_subchapter  = information.get_text(strip = True)
                
                subchapter_inner_chapter_link = information.get('href')
                if subchapter_inner_chapter_link.startswith('..'):
                    subchapter_inner_chapter_link = subchapter_inner_chapter_link[3:]
                subchapter_inner_chapter_response = requests.get(base_link_site + subchapter_inner_chapter_link)
                subchapter_inner_chapter_response.encoding = 'windows-1257'
                subchapter_inner_chapter_soup = BeautifulSoup(subchapter_inner_chapter_response.text, 'lxml')
                if subchapter_inner_chapter_soup.find_all('a'):
                    final_story_links = subchapter_inner_chapter_soup.find_all('a')
                    
                    for final_story_link in final_story_links:
                        story_and_metadata_section_with_details = []
                        metadata_for_3rd_layer = final_story_link.get_text()
                        link_to_final_story_ =  final_story_link.get('href')
                        
                        if link_to_final_story_ is not None:
                            if link_to_final_story_.startswith('..'):
                                link_to_final_story_updated = link_to_final_story_[3:]
                            else:
                                link_to_final_story_updated = link_to_final_story_
                        else:
                            continue
                        
                        if link_to_final_story_updated:
                            full_link_to_final_story_updated = base_link_site + link_to_final_story_updated
                        
                        valoda_ailab_lv_response = requests.get(full_link_to_final_story_updated)
                        valoda_ailab_lv_response.encoding = 'windows-1257'
                        valoda_ailab_lv_response_soup = BeautifulSoup(valoda_ailab_lv_response.text, 'lxml')
                        finding_the_story_last = valoda_ailab_lv_response_soup.find_all('p')
                        filtered_finding_the_story_last = [item for item in finding_the_story_last if item.get_text().strip()]
                        
                        for each_item_in_the_story in filtered_finding_the_story_last: 
                            story_and_metadata_section_with_details.append(each_item_in_the_story.get_text(strip = True))
                      
                         #if len(story_and_metadata_section_with_details[1]) <=100:
#                             metadata = "".join(story_and_metadata_section_with_details[1]).strip()
#                         else: 
#                             metadata = story_and_metadata_section_with_details[:1].strip()
                        title_of_chapter = title.strip()
                        title_of_subchapter  = story_and_metadata_section_with_details[0]
                        url_of_chapter = full_link_to_final_story_updated
                        metadata_of_chapter = metadata_for_3rd_layer
                        full_story_of_chapter = "".join(story_and_metadata_section_with_details[2:]).strip() 
                        add_entry_to_list()
                        story_and_metadata_section_with_details.clear()
                                
                    
                else:
                    final_story_of_the_subchapter = subchapter_inner_chapter_soup.find_all('p')
                    filtered_final_story_of_the_subchapter = [item for item in final_story_of_the_subchapter if item.get_text().strip()]
                    for each_item_in_the_story in filtered_final_story_of_the_subchapter:           
                        story_and_metadata_section_wit_sub.append(each_item_in_the_story.get_text(strip = True))
                        
                    #print(story_and_metadata_section_wit_sub)    
                    # metadata_of_chapter = information.get_text(strip = True)
                    
                    if len(story_and_metadata_section_wit_sub) >= 2:
                        if len(story_and_metadata_section_wit_sub[1]) <=100:
                            metadata_for_2nd_layer = story_and_metadata_section_wit_sub[1]
                        else: 
                            metadata_for_2nd_layer = title_of_subchapter
                    if len(story_and_metadata_section_wit_sub) >= 2:       
                        if  re.search(r'\d', story_and_metadata_section_wit_sub[1]):
                            full_story_of_chapter_temp = "".join(story_and_metadata_section_wit_sub[2:]).strip()
                        else:
                            full_story_of_chapter_temp = "".join(story_and_metadata_section_wit_sub[1:]).strip()
                        
                        
                    title_of_chapter = title.strip()
                    title_of_subchapter  = re.sub(pattern, '', title)
                    url_of_chapter = base_link_site + subchapter_inner_chapter_link
                    metadata_of_chapter = metadata_for_2nd_layer
                    full_story_of_chapter = full_story_of_chapter_temp
                    add_entry_to_list()
                    story_and_metadata_section_wit_sub.clear()
                    
        else:
            informations_about_story = subchapter_soup.find_all('p')
            filtered_informations_about_story = [item for item in informations_about_story if item.get_text().strip()]
            for each_item_in_the_story in filtered_informations_about_story:           
                story_and_metadata_section.append(each_item_in_the_story.get_text(strip = True))
                
            if len(story_and_metadata_section[:2]) <=100:
                metadata = "".join(story_and_metadata_section[:2]).strip()
            else: 
                metadata = story_and_metadata_section[:1].strip()
            title_of_chapter = title.strip()
            url_of_chapter = link
            title_of_subchapter = '-'
            metadata_of_chapter = metadata
            full_story_of_chapter = "".join(story_and_metadata_section[2:]).strip() 
            add_entry_to_list()
      
    df = pd.DataFrame(story_table, columns=['URL', 'Chapter Title', 'Subchapter Title', 'Metadata', 'Text'])
    df.to_excel(f'volume_{inner_list_index + 1}.xlsx', index=False)
            